### <b>Import Packages</b>

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

### <b>Import Data</b>

In [2]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {'data_name': """Test_fields_100821_results.csv""", 'interaction_properties': {}}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

df = itcfs.read_pandas_and_concat(readClient, flightInfo)
df.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,DOWNNAME,TRUEFIELD,YARDSGAINED_Mean,AVG_OVER_TOGO,PLAYIDSTR,RUN_PASS,CONCEPT_8,CONCEPT_9,pred_TD,prob_TD_NO,prob_TD_YES,pred_FIRST,prob_FIRST_NO,prob_FIRST_YES
0,0,0,5,0,480,1st,35,6.36,1.272,100000020,RUN,2,0,N,0.989180609,0.010819391,N,0.562406343,0.437593657
1,0,0,9,-7,1050,1st,45,4.756098,0.528455333,100000054,RUN,5,4,N,0.999278473,0.000721527,Y,0.439295772,0.560704228
2,0,2,4,7,600,3rd,5,8.164835,2.04120875,200000049,PASS,4,9,N,0.929719852,0.070280148,Y,0.391968242,0.608031758
3,0,0,11,7,180,1st,45,6.216667,0.565151545,200000041,PASS,4,1,N,0.999941376,5.86E-05,N,0.676802873,0.323197127
4,0,0,1,0,60,1st,45,13.416667,13.416667,200000085,PASS,8,5,N,0.800382789,0.199617211,N,0.60849475,0.39150525


In [3]:
## To drop variables from a dataframe, simply list the ones you want to keep. Can pass to a new dataframe (as here) or drop from original
df = pd.DataFrame(df, columns=['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'DOWNNAME','TRUEFIELD','RUN_PASS',
       'CONCEPT_8', 'CONCEPT_9','YARDSGAINED_Mean', 'AVG_OVER_TOGO', 'prob_FIRST_YES','pred_FIRST','prob_TD_YES','pred_TD'])

In [4]:
## Re-Name columns as needed by casting new column names using the .columns method. New columns names will be applied in order and must be same length as number of existing columns.
df.columns = ['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'DOWNNAME','TRUEFIELD','RUN_PASS',
       'CONCEPT_8', 'CONCEPT_9','YARDSGAINED_Mean', 'AVG_OVER_TOGO', 'prob_FIRST','TARGET_1ST','prob_TD','TARGET_TD']

In [5]:
df[['HALF','DOWNNUM','YARDSTOGO','SCOREDIFF','TIMELEFT','TRUEFIELD','YARDSGAINED_Mean','AVG_OVER_TOGO',
       'CONCEPT_8','CONCEPT_9','prob_TD','prob_FIRST']] = df[['HALF','DOWNNUM','YARDSTOGO','SCOREDIFF','TIMELEFT','TRUEFIELD','YARDSGAINED_Mean','AVG_OVER_TOGO',
       'CONCEPT_8','CONCEPT_9','prob_TD','prob_FIRST']].apply(pd.to_numeric)

In [6]:
df['PLAYID'] = df['RUN_PASS'] + df['CONCEPT_8'].astype(str)
df['TSHIRT'] = "" 
#    print(len(data))
for i in range(len(df)):
    if df['YARDSTOGO'].iloc[i] <= 3:
        df['TSHIRT'].iloc[i] = "SMALL"
    elif df['YARDSTOGO'].iloc[i] <= 6:
        df['TSHIRT'].iloc[i] = "MEDIUM"
    elif df['YARDSTOGO'].iloc[i] <= 10:
        df['TSHIRT'].iloc[i] = "LONG"  
    elif df['YARDSTOGO'].iloc[i] > 10:
        df['TSHIRT'].iloc[i] = "XLONG" 
    else: df['TSHIRT'].iloc[i] = "NONE" 
        


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
#DataFrame.replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')[source]
df.TARGET_TD = df.TARGET_TD.replace({'N': 0,'Y': 1})
df.TARGET_1ST = df.TARGET_1ST.replace({'N': 0,'Y': 1})


In [8]:
df_TD = pd.DataFrame(df, columns=['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'TRUEFIELD', 'RUN_PASS','YARDSGAINED_Mean', 'AVG_OVER_TOGO','PLAYID','TSHIRT'])
df_TD.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,TRUEFIELD,RUN_PASS,YARDSGAINED_Mean,AVG_OVER_TOGO,PLAYID,TSHIRT
0,0,0,5,0,480,35,RUN,6.360000,1.272000,RUN2,MEDIUM
1,0,0,9,-7,1050,45,RUN,4.756098,0.528455,RUN5,LONG
2,0,2,4,7,600,5,PASS,8.164835,2.041209,PASS4,MEDIUM
3,0,0,11,7,180,45,PASS,6.216667,0.565152,PASS4,XLONG
4,0,0,1,0,60,45,PASS,13.416667,13.416667,PASS8,SMALL


In [46]:
df_TD.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,TRUEFIELD,RUN_PASS,YARDSGAINED_Mean,AVG_OVER_TOGO,PLAYID,TSHIRT
0,0,0,5,0,480,35,RUN,6.360000,1.272000,RUN2,MEDIUM
1,0,0,9,-7,1050,45,RUN,4.756098,0.528455,RUN5,LONG
2,0,2,4,7,600,5,PASS,8.164835,2.041209,PASS4,MEDIUM
3,0,0,11,7,180,45,PASS,6.216667,0.565152,PASS4,XLONG
4,0,0,1,0,60,45,PASS,13.416667,13.416667,PASS8,SMALL


In [9]:
from sklearn.preprocessing import OneHotEncoder

bin_data = pd.get_dummies(df_TD[['RUN_PASS','PLAYID','TSHIRT']]) ## list the fields which you want to encode
bin_data = bin_data.drop(['RUN_PASS_PASS'], axis=1) ## where the target field is binary, first field can be dropped

df_TD = df_TD.drop(['RUN_PASS','PLAYID','TSHIRT'], axis=1) ## drop original fields
df_TD = pd.concat([df_TD, bin_data], axis=1)

Rescaling the input variables creates a fitted scaler based on training data. That sclaer must be saved and used to convert data at runtime.

In [10]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(df_TD.values)
scaler

XTD_scaled = scaler.transform(df_TD.values)

## Create & Deploy Test Function

In [11]:
!pip list | grep watson

ibm-watson-machine-learning 1.0.173
ibm-watson-openscale        3.0.12
ibm-watson-studio-lib       3.0.6


In [12]:
%%writefile tmp_py_function.py

custom_yaml = """
dependencies:
  - pip
  - pip:
    - --index-url https://test.pypi.org/simple/
    - lightgbm==3.3.1
"""

with open("tmp.yaml","w") as f:
    f.write(custom_yaml)

print("tmp.yaml written")


def score(input_data) :
    import subprocess
    
    # ignore input_data
    my_log_msgs = []
    
    runlog = subprocess.run(["conda","env","update","--file","tmp.yaml"], check=False, stdout=subprocess.PIPE,
                              stderr=subprocess.STDOUT, text=True)
    my_log_msgs.append(runlog.stdout)
    try:
        import lightgbm   
    except Exception as exc:
        my_log_msgs.append("import failed")
    
    
    score_response = {
            'predictions': [{ 'fields':["log_msgs"], 
                              'values': [my_log_msgs] }]
    } 
    return score_response


Writing tmp_py_function.py


In [7]:
import tmp_py_function

tmp.yaml written


In [13]:
import os
from ibm_watson_machine_learning import APIClient

wml_credentials = { "url" : os.environ['RUNTIME_ENV_APSX_URL'],        
                    "token": os.environ['USER_ACCESS_TOKEN'],
                    "instance_id" : "openshift", 
                    "version": "4.0"
        }

wml_client = APIClient(wml_credentials)
    
type(wml_client), wml_client.version

(ibm_watson_machine_learning.client.APIClient, '1.0.173')

In [14]:
# Connect to current WStudio project
wml_client.set.default_project(os.environ['PROJECT_ID'])
# should return 'SUCCESS'

'SUCCESS'

In [17]:
wml_client.spaces.list(limit=30)

------------------------------------  -----------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                     CREATED
e02d8142-d472-4b2f-bd48-34d1f278f5bc  ASB_TEST_Deploy_ds                                                       2022-01-13T19:14:18.478Z
08acc5cb-66c3-4aef-a54e-29da16f8c7dc  openscale-express-path-preprod-00000000-0000-0000-0000-1639718812425487  2021-12-17T05:43:27.892Z
a5fe8631-b90b-4365-917a-6e173bb2e164  openscale-express-path-00000000-0000-0000-0000-1639718812425487          2021-12-17T05:43:10.158Z
b3ddf01f-4587-41be-b190-ee899830b5d3  toy-image-model                                                          2021-11-09T20:30:59.655Z
54ac1e3e-cf4b-40d0-b4d9-26c339eddb1c  openscale-express-path-preprod-00000000-0000-0000-0000-000000000000      2021-10-29T20:54:12.711Z
7a8e4077-3a02-4752-915f-26ae0d74160a  openscale-express-path-0000

In [18]:
space_id = "e02d8142-d472-4b2f-bd48-34d1f278f5bc"
wml_client.set.default_space(space_id)

Unsetting the project_id ...


'SUCCESS'

In [19]:
space_id

'e02d8142-d472-4b2f-bd48-34d1f278f5bc'

In [20]:
# copied from wml utilities
def wml_util__store_python_function_file( wml_client, file_name, name,sw_spec_name=None) :
    import sys
    if not sw_spec_name :
        py_version = sys.version[:3]   # 3.8 or 3.7
        if py_version == "3.7":
            sw_spec_name="default_py"+py_version+"_opence"
        else :
            sw_spec_name="default_py"+py_version 
    #
    sw_spec_id = wml_client.software_specifications.get_uid_by_name(sw_spec_name)
    print("sw_spec",sw_spec_name,sw_spec_id)
    assert(sw_spec_id and sw_spec_id != "Not Found")
    meta_props = {
        wml_client.repository.FunctionMetaNames.NAME: name,
        wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID : sw_spec_id
    }
    assert isinstance(file_name,str) , "function_name must be name of the .py file, not the Python function object"
    import gzip
    with open(file_name, 'rb') as orig_file:
        with gzip.open(file_name+".gz", 'wb') as zipped_file:
            zipped_file.writelines(orig_file)
        
    function_artefact = wml_client.repository.store_function(meta_props=meta_props, function=file_name+".gz")
    function_uid = wml_client.repository.get_function_uid(function_artefact)
    # print("Stored Function UID = " + function_uid)
    return function_uid

In [34]:
wml_client.software_specifications.list()

Failure during list sw_specs. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true)
Status code: 500, body: {"trace":"4b3324de-a2db-43dc-bfbb-e712cd3e35e3","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}


ApiRequestFailure: Failure during list sw_specs. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true)
Status code: 500, body: {"trace":"4b3324de-a2db-43dc-bfbb-e712cd3e35e3","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}

In [35]:
# specify the software specification for the model
sw_spec_id = wml_client.software_specifications.get_id_by_name('default_py3.7')
sw_spec_id

Failure during list assets. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true&name=default_py3.7)
Status code: 500, body: {"trace":"e18b8bea-8842-4e7f-9864-0287175b1f6e","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}


ApiRequestFailure: Failure during list assets. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true&name=default_py3.7)
Status code: 500, body: {"trace":"e18b8bea-8842-4e7f-9864-0287175b1f6e","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}

In [21]:
my_wml_function = wml_util__store_python_function_file(wml_client,"tmp_py_function.py","tmp_py_function")

Failure during list assets. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true&name=default_py3.7_opence)
Status code: 500, body: {"trace":"a2a12a34-aa6d-4ceb-8795-44e6780df7a4","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}


ApiRequestFailure: Failure during list assets. (GET https://internal-nginx-svc:12443/v2/software_specifications?version=2021-06-24&space_id=e02d8142-d472-4b2f-bd48-34d1f278f5bc&userfs=true&name=default_py3.7_opence)
Status code: 500, body: {"trace":"a2a12a34-aa6d-4ceb-8795-44e6780df7a4","errors":[{"code":"invalid_response","message":"Invalid error response for search software specification asset. Details: {\"code\":400,\"error\":\"Bad Request\",\"reason\":\"Required request parameters missing\",\"message\":\"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).\"}"}]}

In [22]:
wml_client.repository.list_models()

--  ----  -------  ----
ID  NAME  CREATED  TYPE
--  ----  -------  ----


## Deploy Scaler to space

In [30]:
metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'FIRST_DOWN_SCALER',
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23",
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}

model_info = wml_client.repository.store_model(scaler,
                                               meta_props=metadata,
                                               training_data=df_TD.values,
                                               training_target=df['TARGET_1ST'].values)

model_id = wml_client.repository.get_model_uid(model_info)
model_id

'85c7da71-360f-45a8-a20f-755641f13335'

## Build Model and Deploy to Space

In [31]:
from sklearn.model_selection import train_test_split

## https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

#X_1st = df_TD.values ## We are using the same input variables, so the same scaled data will work. Only change is the target and stratification variables.
y_1st = df['TARGET_1ST'].values

train_X1ST, test_X1ST, train_y1st, test_y1st = train_test_split(XTD_scaled, y_1st, test_size=0.3,stratify=df.STRAT_1ST, random_state=121)

Import and run the Random Forest Classifier using the parameters established in model development phase.

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf_1ST = RandomForestClassifier(max_depth=7, random_state=33)
clf_1ST.fit(train_X1ST,train_y1st)
clf_1ST.score(test_X1ST, test_y1st)

## Deploy the Model

In [38]:
metadata = {
    wml_client.repository.ModelMetaNames.NAME: 'FIRST_DOWN_MODEL',
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23",
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}

model_info = wml_client.repository.store_model(clf_1ST,
                                               meta_props=metadata,
                                               training_data=train_X1ST,
                                               training_target=train_y1st)

model_id = wml_client.repository.get_model_uid(model_info)
model_id

'ddaad323-f7ae-444a-b132-9bbc46b16d83'

In [39]:
wml_client.repository.list_models()

------------------------------------  -----------------  ------------------------  -----------------
ID                                    NAME               CREATED                   TYPE
ddaad323-f7ae-444a-b132-9bbc46b16d83  FIRST_DOWN_MODEL   2022-01-11T23:09:44.002Z  scikit-learn_0.23
85c7da71-360f-45a8-a20f-755641f13335  FIRST_DOWN_SCALER  2022-01-11T22:56:50.002Z  scikit-learn_0.23
------------------------------------  -----------------  ------------------------  -----------------


## Create Function to Transform and Score the Data

In [216]:
def data_process(url=os.environ['RUNTIME_ENV_APSX_URL'],project_id=os.environ['PROJECT_ID'], space_id=space_id, token=os.environ['USER_ACCESS_TOKEN'], payload=payload): #os.environ['SPACE_ID']):
    # May be needed for above definition: token=os.environ['USER_ACCESS_TOKEN'],
    # keep all function arguments in one line to avoid 
    
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import RepeatedStratifiedKFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification
    import json
       
    # Load model from the deployment space
    from ibm_watson_machine_learning import APIClient
    import os
    
    wml_credentials = { "url" : os.environ['RUNTIME_ENV_APSX_URL'],        
                    "token": os.environ['USER_ACCESS_TOKEN'],
                    "instance_id" : "openshift", 
                    "version": "4.0"
        }      

    #NOTE: _fn to refer to the wml_client instance running inside the functiopn
    wml_client_fn = APIClient(wml_credentials)

    #Space ID taken by pre-running cells above
    space_id_fn = '36aa3b6f-2329-4382-aa25-973ccf667470'
    wml_client_fn.set.default_space(space_id_fn)

    # Load scaler from the deployment space
    scaler_obj = wml_client_fn.repository.load('85c7da71-360f-45a8-a20f-755641f13335')
    model_obj = wml_client_fn.repository.load('ddaad323-f7ae-444a-b132-9bbc46b16d83')
        
    def score(payload):   

            #Set client to the function space identified above
            wml_client_fn.set.default_space('36aa3b6f-2329-4382-aa25-973ccf667470')

            outputs =[]
            
            fields = payload["input_data"][0]['fields']

            for i in range(len(payload.get("input_data")[0].get("values"))):
                list_of_records = payload.get("input_data")[0].get("values")[i]
                list_of_fields = payload.get("input_data")[0].get("fields")    

            array_of_records = np.array(list_of_records)
            array_of_records = array_of_records.reshape(1,-1)
            scaled = scaler_obj.transform(array_of_records)

            scored = model_obj.predict(scaled).tolist()[0],model_obj.predict_proba(scaled).tolist()[0]
            outputs.append(scored)    
            
            print(outputs)            
            payload_output = {'predictions':[{'fields': ['prediction','probability']},{'values': outputs}]}
            

            return payload_output

    return score

### Create data payload to pre-test the function

In [85]:
payload = { "input_data": [{ "fields": ['HALF', 'DOWNNUM','YARDSTOGO','SCOREDIFF','TIMELEFT','TRUEFIELD',
'YARDSGAINED_Mean','AVG_OVER_TOGO','RUN_PASS_RUN','PLAYID_PASS4','PLAYID_PASS5','PLAYID_PASS6','PLAYID_PASS7','PLAYID_PASS8',
'PLAYID_PASS9','PLAYID_RUN1','PLAYID_RUN2','PLAYID_RUN3','PLAYID_RUN5','TSHIRT_LONG','TSHIRT_MEDIUM','TSHIRT_SMALL','TSHIRT_XLONG'],
                                "values": [[1,2,10,-5,470,25,5.277778,0.527,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0]]}]}   

In [194]:
print(payload)

{'input_data': [{'fields': ['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'TRUEFIELD', 'YARDSGAINED_Mean', 'AVG_OVER_TOGO', 'RUN_PASS_RUN', 'PLAYID_PASS4', 'PLAYID_PASS5', 'PLAYID_PASS6', 'PLAYID_PASS7', 'PLAYID_PASS8', 'PLAYID_PASS9', 'PLAYID_RUN1', 'PLAYID_RUN2', 'PLAYID_RUN3', 'PLAYID_RUN5', 'TSHIRT_LONG', 'TSHIRT_MEDIUM', 'TSHIRT_SMALL', 'TSHIRT_XLONG'], 'values': [[1, 2, 10, -5, 470, 25, 5.277778, 0.527, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]}]}


In [217]:
import pandas as pd
function_result = data_process()(payload)
print(function_result)

[('N', [0.6091495374437705, 0.3908504625562296])]
{'predictions': [{'fields': ['prediction', 'probability']}, {'values': [('N', [0.6091495374437705, 0.3908504625562296])]}]}


In [176]:
data_process

<function __main__.data_process(url='https://internal-nginx-svc:12443', project_id='a56034f5-5534-4da8-894e-4b7363ffa274', space_id='36aa3b6f-2329-4382-aa25-973ccf667470', payload={'input_data': [{'fields': ['HALF', 'DOWNNUM', 'YARDSTOGO', 'SCOREDIFF', 'TIMELEFT', 'TRUEFIELD', 'YARDSGAINED_Mean', 'AVG_OVER_TOGO', 'RUN_PASS_RUN', 'PLAYID_PASS4', 'PLAYID_PASS5', 'PLAYID_PASS6', 'PLAYID_PASS7', 'PLAYID_PASS8', 'PLAYID_PASS9', 'PLAYID_RUN1', 'PLAYID_RUN2', 'PLAYID_RUN3', 'PLAYID_RUN5', 'TSHIRT_LONG', 'TSHIRT_MEDIUM', 'TSHIRT_SMALL', 'TSHIRT_XLONG'], 'values': [[1, 2, 10, -5, 470, 25, 5.277778, 0.527, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]}]})>

### Store the function in the Deployment Space

In [218]:
sofware_spec_uid_function = wml_client.software_specifications.get_id_by_name("default_py3.7_opence")
sofware_spec_uid_function

'c2057dd4-f42c-5f77-a02f-72bdbd3282c9'

In [219]:
function_meta_props = {
         wml_client.repository.FunctionMetaNames.NAME: 'data_process',
         wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: sofware_spec_uid_function
         }

In [220]:
function_meta_props

{'name': 'data_process',
 'software_spec_id': 'c2057dd4-f42c-5f77-a02f-72bdbd3282c9'}

In [221]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=data_process)

Function is now stored and can be called from deployment space.

In [222]:
wml_client.repository.list_functions()

------------------------------------  ---------------  ------------------------  ------
GUID                                  NAME             CREATED                   TYPE
4bda823b-d151-4e5d-81e2-4fa5432e70e4  data_process     2022-01-12T23:11:13.002Z  python
b1d1375b-d03a-4bcc-b746-8e1f6693d88a  tmp_py_function  2022-01-11T22:46:34.002Z  python
------------------------------------  ---------------  ------------------------  ------


In [223]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=data_process)
function_uid = wml_client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

Function UID = ad1e5f19-d3de-4a60-9ec8-fb17cfb3c820


## Creating the Deployment

In [208]:
wml_client.deployments.list()

----  ----  -----  -------
GUID  NAME  STATE  CREATED
----  ----  -----  -------


In [ ]:
## Show selection of hardware specifications and tune for job size and priority
#wml_client.hardware_specifications.list()

In [224]:
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('S')

In [225]:
function_deploy_meta = {
     wml_client.deployments.ConfigurationMetaNames.NAME: "data_process",
    ## Next line passes an empty dictionary for the payload in an online deployment
     wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
     wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
 }

In [214]:
import os
token = os.environ['USER_ACCESS_TOKEN']

In [215]:
token

'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ino1S0lLNGZmZ3RPVTJ4R2ZMR0RJNFJQakFCWWNYbURrWk1sclBoeVpwMFEifQ.eyJ1aWQiOiIxMDAwMzMxMTExIiwidXNlcm5hbWUiOiJkYW4uc2NoZXVjaEB1cy5pYm0uY29tIiwicm9sZSI6IlVzZXIiLCJwZXJtaXNzaW9ucyI6WyJjcmVhdGVfcHJvamVjdCIsImFjY2Vzc19jYXRhbG9nIiwibW9uaXRvcl9zcGFjZSIsInZpZXdfcGxhdGZvcm1faGVhbHRoIiwic2hhcmVfc2VjcmV0cyIsImF1dGhvcl9nb3Zlcm5hbmNlX2FydGlmYWN0cyIsInZpZXdfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJ2aWV3X3F1YWxpdHkiLCJtYW5hZ2VfaW5mb3JtYXRpb25fYXNzZXRzIiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsIm1hbmFnZV9kaXNjb3ZlcnkiLCJtYW5hZ2VfcXVhbGl0eSIsImNyZWF0ZV9zcGFjZSIsImFjY2Vzc19hZHZhbmNlZF9nb3Zlcm5hbmNlX2NhcGFiaWxpdGllcyIsImFjY2Vzc19hZHZhbmNlZF9tYXBwaW5nX2NhcGFiaWxpdGllcyIsImFjY2Vzc19pbmZvcm1hdGlvbl9hc3NldHMiLCJtYW5hZ2VfY2F0ZWdvcmllcyIsInZpcnR1YWxpemVfdHJhbnNmb3JtIiwibWFuYWdlX2NhdGFsb2ciXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJkYW4uc2NoZXVjaEB1cy5pYm0uY29tIiwiaXNzIjoiS05PWFNTTyIsImF1ZCI6IkRTWCIsImlhdCI6MTY0MjAwMjY3MCwiZXhwIjozNjE2NDE5OTkwNzB9.LCcnzk-EYqwi4J9Be22HaJw9LxURT_29WpELyGqbJv0nJ0o

In [ ]:
meta_props = {

wml_client.deployments.ConfigurationMetaNames.NAME: "SAMPLE DEPLOYMENT NAME",

wml_client.deployments.ConfigurationMetaNames.ONLINE: {},

wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC : { "id":  "e7ed1d6c-2e89-42d7-aed5-8sb972c1d2b"}

}

In [ ]:
deployment_details = client.deployments.create(artifact_uid, meta_props)

In [226]:
function_deployment_details = wml_client.deployments.create(function_uid, meta_props=function_deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'ad1e5f19-d3de-4a60-9ec8-fb17cfb3c820' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
...

Deployment creation failed for deployment id: 60e7ef8e-92e0-45f7-beef-2b6d89df6820. Errors: [{'code': 'load_model_failure', 'message': "'USER_ACCESS_TOKEN'", 'target': {'type': 'none', 'name': 'none'}, 'more_info': 'none'}]



failed

--------------------------
Deployment creation failed
--------------------------


'USER_ACCESS_TOKEN'


WMLClientError: Deployment creation failed for deployment id: 60e7ef8e-92e0-45f7-beef-2b6d89df6820. Errors: [{'code': 'load_model_failure', 'message': "'USER_ACCESS_TOKEN'", 'target': {'type': 'none', 'name': 'none'}, 'more_info': 'none'}]